## First Time Installation

Install the latest version of Vertex SDK for Python. First time in instance only.

In [ ]:
# # Google Cloud Notebook
# import os
# if os.path.exists("/opt/deeplearning/metadata/env_version"):
#     USER_FLAG = "--user"
# else:
#     USER_FLAG = ""

# ! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG
# ! pip3 install -U google-cloud-storage $USER_FLAG
# ! pip3 install $USER kfp google-cloud-pipeline-components --upgrade
# ! pip3 install $USER icecream==2.1.1 --upgrade
# ! pip3 install $USER pandas-gbq==0.15.0 --upgrade
# ! pip3 install $USER google-cloud-secret-manager --upgrade

# if not os.getenv("IS_TESTING"):
#     # Automatically restart kernel after installs
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

# # Check versions
# ! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
# ! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

## Load your pipeline components and configs

In [ ]:
%load_ext autoreload
%autoreload 2

# Related Python packages
from icecream import ic
from kfp.v2 import dsl

# Components
from ml_components.dataimport import (
    get_rundates,
    bq_query_no_return,
)
from ml_components.alert import (
    push_slack_notification,
)
from ml_components.pipelinehelper import (
    func_op,
    save_pipeline,
    run_pipeline,
)

# Configs
from config import (
    SERVICE_ACCOUNT,
    PROJECT_ID,
    REGION,
    RUNNER,
    PIPELINE_NAME,
    BUCKET_NAME,
    DATAMODEL_PIPELINE_ROOT as PIPELINE_ROOT,
    PARAMETER_VALUES,
    parameter_checks
)
parameter_checks()

## Define your pipeline
```
@dsl.pipeline(
    name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
)
def your_pipeline(
    parameter_value: data_type,
):
    ops_1 = func_op(
        func=function,
        _component_human_name='your ops 1 label',
        base_image='python:3.7', #optional
        function_arg=parameter_value,
    )
    
    ops_2 = func_op(
        func=function,
        _component_human_name='your ops 2 label',
        packages_to_install=[
            'pandas==1.3.3'
        ],
        function_arg=ops_1.outputs['output'],
    )
```

#### Choose your desired GCP pre-built image for your node(s)
- https://cloud.google.com/deep-learning-containers/docs/choosing-container
- `! gcloud container images list --repository="gcr.io/deeplearning-platform-release"`
- You can also just choose 'python:3.x' if want

In [ ]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
)
def grand_pipeline(
    bucket_name: str,
    project_id: str,
    job_id: str,
    run_date: str,
    data_features_dict: str,
    current_drift_threshold: float,
    train_drift_threshold: float,
    pred_drift_threshold: float,
    model_params: str,
    failure_webhook_config: str,
    progress_webhook_config: str,
    runner: str,
):
    ########################################################################################################################
    ########################################################################################################################
    ################################################### Ops Declaration ####################################################
    ########################################################################################################################
    ########################################################################################################################

    
    ###########################################################
    ######################### Rundate #########################
    ###########################################################

    rundates_op = func_op(
        func=get_rundates,
        _component_human_name='get_rundates',
        base_image='python:3.7',
        packages_to_install=['pandas==1.3.3'],
        run_date=run_date,
    )

    usage_run_date = rundates_op.outputs['usage_run_date']
    src_run_dt = rundates_op.outputs['src_run_dt']

    ############
    #### L1 ####
    ############

    l1_01 = 'stored_proc_l1'
    query_l2_01 = f"""
    CALL `{project_id}.PROCEDURES.{l1_01}`("{usage_run_date}");
    """
    l1_01_op = func_op(
        func=bq_query_no_return,
        _component_human_name=l1_01,
        base_image='python:3.7',
        project_id=project_id,
        query=query_l2_01,
    )


    ############
    #### L2 ####
    ############

    l2_01 = 'stored_proc_l2'
    query_l2_01 = f"""
    CALL `{project_id}.PROCEDURES.{l2_01}`("{usage_run_date}");
    """
    l2_01_op = func_op(
        func=bq_query_no_return,
        _component_human_name=l2_01,
        base_image='python:3.7',
        project_id=project_id,
        query=query_l2_01,
    )


    ############
    #### L3 ####
    ############

    l3_01 = 'stored_proc_l3'
    query_l3_01 = f"""
    CALL `{project_id}.PROCEDURES.{l3_01}`("{usage_run_date}");
    """
    l3_01_op = func_op(
        func=bq_query_no_return,
        _component_human_name=l3_01,
        base_image='python:3.7',
        project_id=project_id,
        query=query_l3_01,
    )


    ########################################################################################################################
    ########################################################################################################################
    ####################################################### Ops Flow #######################################################
    ########################################################################################################################
    ########################################################################################################################


    ###########################################################
    ################### Datamodel Generation ##################
    ###########################################################

    l1_01_op.after(rundates_op)
    l2_01_op.after(l1_01_op)
    l3_01_op.after(l2_01_op)


    ########################################################################################################################
    ########################################################################################################################
    ##################################################### Ops Caching ######################################################
    ########################################################################################################################
    ########################################################################################################################

    nodes_no_cache = [
        rundates_op,
        # l3_03_op,
    ]
    
    if nodes_no_cache:
        for node in nodes_no_cache:
            node.set_caching_options(enable_caching=False)

## Save your pipeline

In [ ]:
TEMPLATE_PATH, JOB_ID, DISPLAY_NAME = save_pipeline(
    pipeline=grand_pipeline,
    pipeline_name=PIPELINE_NAME,
    bucket_name=BUCKET_NAME,
    mode='datamodel',
)

## Run your pipeline locally

In [ ]:
# run_pipeline(
#     project_id=PROJECT_ID,
#     staging_bucket=BUCKET_NAME,
#     location=REGION,
#     display_name=PIPELINE_NAME,
#     template_path=TEMPLATE_PATH,
#     job_id=JOB_ID,
#     pipeline_root=PIPELINE_ROOT,
#     service_account=SERVICE_ACCOUNT,
#     parameter_values=PARAMETER_VALUES,
#     disable_caching_all=True,
# )